In [ ]:
import os
import polars as pl
import pandas as pd
import openpyxl
from openpyxl import Workbook
from pandas import ExcelWriter

In [ ]:
Documentos = [
    "A_agri.xlsx",
    "B_mining.xlsx",
    "C10_food.xlsx",
    "C13_textil.xlsx",
    "C16_wood.xlsx",
    "C17_paper.xlsx",
    "C18_printing.xlsx",
    "C19_petroleum.xlsx",
    "C22_plastic.xlsx",
    "C24_metals.xlsx",
    "D351_electricity.xlsx",
    "H_transport.xlsx",
    "Recycled.xlsx"
]

Material = [
    "aluminium",
    "Copper",
    "Lead",
    "Zinc",
    "Nickel",
    "Polyethylene", 
    "Polypropylene",
    "Polyvinyl chloride",
    "Cement",
    "Concrete",
    "Glass",
    "Wood"
]

Categorias = [
    "Material",
    "Acidification: terrestrial",
    "Climate change",
    "Ecotoxicity: freshwater",
    "Ecotoxicity: marine",
    "Ecotoxicity: terrestrial",
    "Energy resources: non-renewable, fossil",
    "Eutrophication: freshwater",
    "Eutrophication: marine",
    "Human toxicity: carcinogenic",
    "Human toxicity: non-carcinogenic",
    "Ionising radiation",
    "Land use",
    "Material resources: metals/minerals",
    "Ozone depletion",
    "Particulate matter formation",
    "Photochemical oxidant formation: human health",
    "Photochemical oxidant formation: terrestrial ecosystems",
    "Water use"
]

In [ ]:
output_file = "Materiales.xlsx"

# Crea el archivo 
if not os.path.exists(output_file):
    wb = Workbook()
    # Guardar el archivo
    wb.save(output_file)
    print(f"Archivo '{output_file}' creado con éxito.")
else:
    print(f"El archivo '{output_file}' ya existe, no se creó uno nuevo.")
# Liberar el archivo 
wb = openpyxl.load_workbook(output_file)
wb.close()


In [ ]:
for i in Documentos:
    df = pl.read_excel(i, has_header=True)
    df = df.rename(dict(zip(df.columns, Categorias)))

    # Precomputar versión en minúsculas de la col. Material
    col_lower = pl.col("Material").cast(pl.Utf8).str.to_lowercase()

    for mat in Material:
        patt = mat.lower() 

        # Filtrar coincidencias literales 
        df_filtered = df.filter(col_lower.str.contains(patt, literal=True))

        if df_filtered.height > 0:
            #df_filtered = df_filtered.with_columns(pl.lit(i).alias("Fuente"))

            # Convertir a pandas para escribir al Excel
            new_pd = df_filtered.to_pandas()
            sheet_name = mat.capitalize()

            # Leer contenido existente
            try:
                existing_pd = pd.read_excel(output_file, sheet_name=sheet_name )#(#)#
                combined = pd.concat([existing_pd, new_pd], ignore_index=True) #pd 
                header=False ######
            except Exception:
                combined = new_pd
                write_header = True ########

            # Escribir reemplazando la hoja por la versión combinada

            with ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
                combined.to_excel(writer, sheet_name=sheet_name, index=False, header=write_header) # header=False

            print(f"{sheet_name}: +{len(new_pd)} filas desde {i}")

# Liberar el archivo 
wb = openpyxl.load_workbook(output_file)
wb.close()

print("✅ Búsqueda completada, datos acumulados y archivo liberado.")